优化矩阵乘法

矩阵乘法是计算密集型运算，为了取得良好的CPU性能，有两个重要的优化
+ 提高内存访问的Cache命中率，需要将原始内存访问模式转换为适合缓存策略的模式，提高局部性
+ SIMD（单指令多数据），向量处理单元，在每个循环中处理一小批数据而不是处理单个值，需要将循环体中的数据访问模式转换为统一模式，以便LLVM后端可以将其降低到SIMD

In [2]:
import tvm
import tvm.testing
from tvm import te
import numpy
import timeit

# 矩阵的大小
# (M, K) x (K, N)
# 可尝试不同的 shape，TVM 优化的性能有时比 numpy + MKL 更好
M = 1024
K = 1024
N = 1024


# TVM 默认张量数据类型
dtype = "float32"

# 你可能想调整 target 使其和你的任何 CPU 向量扩展匹配
# 例如，如果你为 SIMD 用的是 Intel AVX2（高级向量扩展）ISA，把下面这行换成 `llvm -mcpu=core-avx2` 可以取得最佳性能（或者你所用 CPU 的具体类型）
# 记住你用的是 llvm, 可以用 `llc --version` 命令来获取 CPU 类型，也可以查看 `/proc/cpuinfo` 来获取你处理器支持的更多扩展

target = tvm.target.Target(target="llvm", host="llvm")
dev = tvm.device(target.kind.name, 0)

# 为测试随机生成的张量
a = tvm.nd.array(numpy.random.rand(M, K).astype(dtype), dev)
b = tvm.nd.array(numpy.random.rand(K, N).astype(dtype), dev)

# 重复执行矩阵乘法以获得默认 numpy 实现的性能基线
np_repeat = 100
np_running_time = timeit.timeit(
    setup="import numpy\n"
    "M = " + str(M) + "\n"
    "K = " + str(K) + "\n"
    "N = " + str(N) + "\n"
    'dtype = "float32"\n'
    "a = numpy.random.rand(M, K).astype(dtype)\n"
    "b = numpy.random.rand(K, N).astype(dtype)\n",
    stmt="answer = numpy.dot(a, b)",
    number=np_repeat,
)
print("Numpy running time: %f" % (np_running_time / np_repeat))

answer = numpy.dot(a.numpy(), b.numpy())

Numpy running time: 0.006067


用TVM TE编写一个基本的矩阵乘法，并验证它是否产生与numpy实现相同的结果，在探索性能

In [3]:
# 用 TE 的 TVM 矩阵乘法
k = te.reduce_axis((0, K), "k")
A = te.placeholder((M, K), name="A")
B = te.placeholder((K, N), name="B")
C = te.compute((M, N), lambda x, y: te.sum(A[x, k] * B[k, y], axis=k), name="C")

# 默认 schedule
s = te.create_schedule(C.op)
func = tvm.build(s, [A, B, C], target=target, name="mmult")

c = tvm.nd.array(numpy.zeros((M, N), dtype=dtype), dev)
func(a, b, c)
tvm.testing.assert_allclose(c.numpy(), answer, rtol=1e-5)

def evaluate_operation(s, vars, target, name, optimization, log):
    func = tvm.build(s, [A, B, C], target=target, name="mmult")
    assert func

    c = tvm.nd.array(numpy.zeros((M, N), dtype=dtype), dev)
    func(a, b, c)
    tvm.testing.assert_allclose(c.numpy(), answer, rtol=1e-5)

    evaluator = func.time_evaluator(func.entry_name, dev, number=10)
    mean_time = evaluator(a, b, c).mean
    print("%s: %f" % (optimization, mean_time))
    log.append((optimization, mean_time))

log = []

evaluate_operation(s, [A, B, C], target=target, name="mmult", optimization="none", log=log)

[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[17:18:41] /home/patrick/Code/tvm/src/ir/transform.cc:440

none: 2.438024
